In [2]:
import os
import sys
import gc
import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from multiprocessing import cpu_count, Pool

In [2]:
train = pd.read_csv('../input/train.csv')

In [3]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [4]:
train['feature_4'] = train['feature_3']

In [5]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,feature_4
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,1
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0


In [32]:
drop = []
columns = train.columns
for i in tqdm(range(0, len(columns))):
    for j in range(i+1, len(columns)):
        if sum(train[columns[i]] != train[columns[j]]) == 0:
            drop.append(columns[j])

100%|██████████| 7/7 [00:00<00:00, 20.09it/s]


In [33]:
drop

['feature_4']

In [3]:
NTHREAD = cpu_count()

PATH = os.path.join('..', 'feature')

KEY = 'card_id'

In [4]:
files = os.listdir(PATH)

test = pd.read_csv(os.path.join('..', 'data', 'test.csv'))

features =  ['f103.pkl', 'f105.pkl', 'f109.pkl']
features += ['f107_N.pkl', 'f107_Y.pkl', 'f108_N.pkl', 'f108_Y.pkl']
features += ['f203.pkl', 'f205.pkl', 'f209.pkl']
features += ['f207_N.pkl', 'f207_Y.pkl', 'f208_N.pkl', 'f208_Y.pkl']
features += ['f302.pkl', 'f303.pkl', 'f304.pkl', 'f305.pkl', 'f306.pkl']
features += ['f403.pkl', 'f404.pkl', 'f409.pkl', 'f410.pkl', 'f411.pkl']
features += ['f406_N.pkl', 'f406_Y.pkl', 'f408_N.pkl', 'f408_Y.pkl']

for f in features:
    print(f'Merge: {f}', end=' ')
    test = pd.merge(test, pd.read_pickle(os.path.join('..', 'feature', f)), on=KEY, how='left')
    print('Done!!')

Merge: f103.pkl Done!!
Merge: f105.pkl Done!!
Merge: f109.pkl Done!!
Merge: f107_N.pkl Done!!
Merge: f107_Y.pkl Done!!
Merge: f108_N.pkl Done!!
Merge: f108_Y.pkl Done!!
Merge: f203.pkl Done!!
Merge: f205.pkl Done!!
Merge: f209.pkl Done!!
Merge: f207_N.pkl Done!!
Merge: f207_Y.pkl Done!!
Merge: f208_N.pkl Done!!
Merge: f208_Y.pkl Done!!
Merge: f302.pkl Done!!
Merge: f303.pkl Done!!
Merge: f304.pkl Done!!
Merge: f305.pkl Done!!
Merge: f306.pkl Done!!
Merge: f403.pkl Done!!
Merge: f404.pkl Done!!
Merge: f409.pkl Done!!
Merge: f410.pkl Done!!
Merge: f411.pkl Done!!
Merge: f406_N.pkl Done!!
Merge: f406_Y.pkl Done!!
Merge: f408_N.pkl Done!!
Merge: f408_Y.pkl Done!!


In [5]:
test.dtypesypes

first_active_month                                   object
card_id                                              object
feature_1                                             int64
feature_2                                             int64
feature_3                                             int64
mean_feature_1                                      float64
mean_feature_2                                      float64
mean_feature_3                                      float64
first_active_month_year                               int64
first_active_month_weekday                            int64
first_active_month_month                              int64
first_active_month_dayofweek                          int64
first_active_month_weekofyear                         int64
elapsed_time                                          int64
hist_month_lag_mean_x                               float16
hist_month_lag_std_x                                float16
hist_month_lag_var                      

In [8]:
str(test['first_active_month'].dtype)

'object'

In [13]:
str(test['feature_1'].dtype)

'int64'

In [17]:
for f in ['hist_purchase_date_max', 'hist_purchase_date_min', 
          'N_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_min', 
          'Y_hist_auth_purchase_date_max', 'Y_hist_auth_purchase_date_min', 
          'new_purchase_date_max', 'new_purchase_date_min', 
          'N_new_auth_purchase_date_max', 'N_new_auth_purchase_date_min',
          'Y_new_auth_purchase_date_max', 'Y_new_auth_purchase_date_min', 
          'union_purchase_date_max', 'union_purchase_date_min', 
          'N_union_auth_purchase_date_max', 'N_union_auth_purchase_date_min',
          'Y_union_auth_purchase_date_max', 'Y_union_auth_purchase_date_min']:
    test[f] = test[f].astype(np.int64) * 1e-9

In [15]:
test = test.drop(['N_authorized_flag_x', 'Y_authorized_flag_x',
                  'N_authorized_flag_y', 'Y_authorized_flag_y',
                  'union_transactions_count_x', 'union_transactions_count_y'], axis=1)

In [ ]:
drop = []
columns = test.columns
for i in tqdm(range(0, len(columns))):
    if str(test[columns[i]].dtype) == 'object':
        continue
    for j in range(i+1, len(columns)):
        if str(test[columns[j]].dtype) == 'object':
            continue
        if sum(test[columns[i]] != test[columns[j]]) == 0:
            drop.append(columns[j])
            print(columns[j])
    del test[columns[i]]



  0%|          | 0/970 [00:00<?, ?it/s]

  0%|          | 3/970 [00:08<43:17,  2.69s/it]

  0%|          | 4/970 [00:16<1:09:30,  4.32s/it]

  1%|          | 5/970 [00:24<1:28:44,  5.52s/it]

  1%|          | 6/970 [00:32<1:41:12,  6.30s/it]

  1%|          | 7/970 [00:40<1:50:07,  6.86s/it]

  1%|          | 8/970 [00:48<1:51:44,  6.97s/it]

  1%|          | 9/970 [00:54<1:48:32,  6.78s/it]

first_active_month_dayofweek




  1%|          | 10/970 [00:58<1:38:06,  6.13s/it]

  1%|          | 11/970 [01:03<1:32:27,  5.78s/it]

  1%|          | 12/970 [01:10<1:36:14,  6.03s/it]

  1%|▏         | 13/970 [01:19<1:48:09,  6.78s/it]

  1%|▏         | 14/970 [01:27<1:55:16,  7.23s/it]

  2%|▏         | 15/970 [01:36<2:02:23,  7.69s/it]

  2%|▏         | 16/970 [01:43<2:01:27,  7.64s/it]

  2%|▏         | 17/970 [01:51<2:00:58,  7.62s/it]

  2%|▏         | 18/970 [01:58<2:01:36,  7.66s/it]

  2%|▏         | 19/970 [02:07<2:04:32,  7.86s/it]

  2%|▏         | 20/970 [02:16<2:10:13,  8.23s/it]

  2%|▏         | 21/970 [02:24<2:10:24,  8.24s/it]

  2%|▏         | 22/970 [02:33<2:13:28,  8.45s/it]

  2%|▏         | 23/970 [02:42<2:13:23,  8.45s/it]

  2%|▏         | 24/970 [02:50<2:11:45,  8.36s/it]

  3%|▎         | 25/970 [02:58<2:11:31,  8.35s/it]

  3%|▎         | 26/970 [03:07<2:13:54,  8.51s/it]

  3%|▎         | 27/970 [03:15<2:12:29,  8.43s/it]

  3%|▎         | 28/970 [03:23<2:10:04,  8.28s/it]

  3%|▎    

hist_installments_count_mean




  5%|▍         | 47/970 [06:50<2:57:32, 11.54s/it]

hist_installments_count_std




  5%|▍         | 48/970 [06:59<2:47:56, 10.93s/it]

hist_installments_count_var




  5%|▌         | 49/970 [07:08<2:38:24, 10.32s/it]

In [ ]:
drop